In [1]:
import os
import pandas as pd
import geopandas as gpd

# Get data from Hidden Dimension Project

In [2]:
data = pd.read_excel(os.path.join("..", "..", "Raw", "HiddenDimensions", "HD Cambodia.xls"))
data['fishing_impact'] = data[[col for col in data.columns if col[:10]=='COAST_fish']].sum(axis=1)
col = {
    'OBJECTID_GADMadm2':"hd_district_id",
    "FOR_G2_loss_area":"forest_loss_area",
    "fishing_impact":"fishing_impact"
}
data = data[list(col.keys())].rename(columns=col)

data.head()

,hd_district_id,forest_loss_area,fishing_impact
0,10719,122.591573,0.0
1,10720,12.361511,0.0
2,10721,5.325560,0.0
3,10722,1.185750,0.0
4,10723,6.044269,0.0


In [3]:
shp = gpd.read_file(os.path.join("..", "..", "Raw", "HiddenDimensions", "Cambodia_gadm2.shp"))
col = {
    'OBJECTID':"hd_district_id", 
    'NAME_2':"hd_district_name", 
    "geometry":"geometry"
}
shp = shp[list(col.keys())].rename(columns=col)
shp['hd_district_area'] = shp.area
print("CRS:", shp.crs)
shp.head()

CRS: {'init': 'epsg:4326'}


,hd_district_id,hd_district_name,geometry,hd_district_area
0,10719,Malai,"POLYGON ((102.818977355957 13.48071956634527, ...",0.047431
1,10720,Mongkol Borei,"POLYGON ((103.0876922607421 13.56660175323481,...",0.059594
2,10721,Ou Chrov,"POLYGON ((102.7456359863282 13.77264022827148,...",0.070253
3,10722,Phnum Srok,"POLYGON ((103.3761367797852 13.94947242736828,...",0.063656
4,10723,Preah Netr Preah,"POLYGON ((103.2661361694337 13.6773824691773, ...",0.081787


In [4]:
# Merge shp and data to have the data of interest as shapefile
data_shp = shp.merge(data, on='hd_district_id', how='left')
data_shp['forest_loss_per_area_unit'] = data_shp['forest_loss_area']/data_shp['hd_district_area']
data_shp['fishing_impact_per_area_unit'] = data_shp['fishing_impact']/data_shp['hd_district_area']
data_shp.head()

,hd_district_id,hd_district_name,geometry,hd_district_area,forest_loss_area,fishing_impact,forest_loss_per_area_unit,fishing_impact_per_area_unit
0,10719,Malai,"POLYGON ((102.818977355957 13.48071956634527, ...",0.047431,122.591573,0.0,2584.622045,0.0
1,10720,Mongkol Borei,"POLYGON ((103.0876922607421 13.56660175323481,...",0.059594,12.361511,0.0,207.429827,0.0
2,10721,Ou Chrov,"POLYGON ((102.7456359863282 13.77264022827148,...",0.070253,5.325560,0.0,75.805529,0.0
3,10722,Phnum Srok,"POLYGON ((103.3761367797852 13.94947242736828,...",0.063656,1.185750,0.0,18.627479,0.0
4,10723,Preah Netr Preah,"POLYGON ((103.2661361694337 13.6773824691773, ...",0.081787,6.044269,0.0,73.902870,0.0


In [58]:
#data_shp.to_file(os.path.join(folder, "data", "Structured", "Supply", "FishingForestry", "hd_fishing_forestry.shp"))

# Not same definition of districts: spatial join with districts as in admin data

In [5]:
admin = gpd.read_file(os.path.join("..", "..", "Structured", "Admin", "level2_2013.shp"))
print("CRS:", admin.crs)
print("Nb district in admin data:", admin.shape[0])
print("Nb district in hd data:", data.shape[0])

CRS: {'init': 'epsg:4326'}
Nb district in admin data: 193
Nb district in hd data: 178


In [11]:
#Slow!
# Spatial join
merged = gpd.overlay(admin[['district_c', 'district_n', 'geometry']], data_shp, how="intersection")
print("CRS:", merged.crs)

# Compute impact on each interesct based on area
merged['intersect_area'] = merged.area
merged['forest_loss_area'] = merged['forest_loss_per_area_unit']*merged['intersect_area']
merged['fishing_impact'] = merged['fishing_impact_per_area_unit']*merged['intersect_area']

# Groupby district
merged = merged[['district_c', 'forest_loss_area', 'fishing_impact', 'intersect_area']]
merged = merged.groupby('district_c').sum()

CRS: None


In [8]:
fishing_forestry = admin[['district_c', 'district_n', 'geometry']].merge(merged, on='district_c', how='left')
filename = os.path.join("..", "..", "Structured", "Supply", "FishingForestry", "fishing_forestry.shp")
fishing_forestry.to_file(filename)
fishing_forestry.head()

NameError: name 'merged' is not defined

# Export Forestry per district

In [11]:
fishing_forestry.set_index('district_c')[['forest_los']].to_csv(os.path.join('..', "..", "Structured", "Supply", "FishingForestry", "forestry.csv"))